In [2]:
import pandas as pd
import numpy as np
import csv
from datetime import datetime, timedelta
from sklearn.datasets import make_regression
from sklearn import linear_model
from sklearn.neighbors import KNeighborsRegressor
from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import statsmodels.api as sm
import matplotlib
import itertools
from pylab import rcParams
import pickle
import warnings
warnings.filterwarnings("ignore")

In [18]:
mapping = {}
for i in range(1,7):=
    file_path = 'low_freq/house_' + str(i) + '/labels.dat'
    mapping[i] = {}
    with open(file_path) as file:
        for line in file:
            mapping[i][int(line.split(' ')[0].strip())] = line.split(' ')[1].strip() + '_' + line.split(' ')[0].strip()
            

'bathroom_gfi_7'

In [99]:
df = {}
for i in range(1,7):
    file_path = 'low_freq/house_' + str(i) + '/channel_' + str(i) + '.dat'
    df[i] = pd.read_csv(file_path , sep = ' ', names = ['TimeStamp', mapping[i][1]], dtype = {'TimeStamp': 'int64', mapping[i][1]:'float64'})
    for j in range(2, len(mapping[i])+1):
        curr_file_path =  'low_freq/house_' + str(i) + '/channel_' + str(j) + '.dat'
        curr = pd.read_csv(curr_file_path , sep = ' ', names = ['TimeStamp', mapping[i][j]], dtype = {'TimeStamp': 'int64', mapping[i][j]:'float64'})
        df[i] = pd.merge(df[i], curr, how='inner', on='TimeStamp')
    df[i]['TimeStamp'] = df[i]['TimeStamp'].astype("datetime64[s]")
        

In [104]:
for i in range(1,7):
    for j, row in df[i].iterrows():
        df[i].at[j,'Year'] = row[0].year
        df[i].at[j,'Month'] = row[0].month
        df[i].at[j,'Day'] = row[0].day
        df[i].at[j,'Hour'] = row[0].hour
        df[i].at[j,'Minute'] = row[0].minute
        df[i].at[j,'Seconds'] = row[0].second
        if row[0].weekday() in [5,6] :
            df[i].at[j, 'Holiday'] = 1
        else:
            df[i].at[j, 'Holiday'] = 0
        if row[0].hour in [0,1,2,3,4,5,6,7,20,21,22,23]:
            df[i].at[j, 'Peak Hour'] = 1
        else:
            df[i].at[j, 'Peak Hour'] = 0
        


,TimeStamp,mains_1,mains_2,oven_3,oven_4,refrigerator_5,dishwaser_6,kitchen_outlets_7,kitchen_outlets_8,lighting_9,...,washer_dryer_19,washer_dryer_20,Year,Month,Day,Hour,Minute,Seconds,Holiday,Peak Hour
0,2011-04-18 13:22:13,222.20,118.83,0.0,0.0,6.0,0.0,34.0,21.0,81.0,...,0.0,0.0,2011.0,4.0,18.0,13.0,22.0,13.0,0.0,0.0
1,2011-04-18 13:22:16,223.17,119.19,0.0,0.0,6.0,0.0,34.0,21.0,81.0,...,0.0,0.0,2011.0,4.0,18.0,13.0,22.0,16.0,0.0,0.0
2,2011-04-18 13:22:20,223.60,118.92,0.0,0.0,6.0,0.0,34.0,22.0,81.0,...,0.0,0.0,2011.0,4.0,18.0,13.0,22.0,20.0,0.0,0.0
3,2011-04-18 13:22:23,222.91,119.16,0.0,0.0,6.0,1.0,35.0,21.0,81.0,...,0.0,0.0,2011.0,4.0,18.0,13.0,22.0,23.0,0.0,0.0
4,2011-04-18 13:22:26,222.94,118.83,0.0,0.0,6.0,0.0,34.0,21.0,81.0,...,0.0,0.0,2011.0,4.0,18.0,13.0,22.0,26.0,0.0,0.0
5,2011-04-18 13:22:30,222.90,118.76,0.0,0.0,6.0,0.0,33.0,21.0,81.0,...,0.0,0.0,2011.0,4.0,18.0,13.0,22.0,30.0,0.0,0.0
6,2011-04-18 13:22:33,222.96,118.88,0.0,0.0,6.0,0.0,34.0,21.0,81.0,...,0.0,0.0,2011.0,4.0,18.0,13.0,22.0,33.0,0.0,0.0
7,2011-04-18 13:22:37,222.54,118.36,0.0,0.0,6.0,0.0,34.0,21.0,82.0,...,0.0,0.0,2011.0,4.0,18.0,13.0,22.0,37.0,0.0,0.0
8,2011-04-18 13:22:40,226.03,119.17,0.0,0.0,6.0,0.0,38.0,21.0,82.0,...,0.0,0.0,2011.0,4.0,18.0,13.0,22.0,40.0,0.0,0.0
9,2011-04-18 13:22:44,222.96,119.03,0.0,0.0,6.0,0.0,34.0,21.0,81.0,...,0.0,0.0,2011.0,4.0,18.0,13.0,22.0,44.0,0.0,0.0


In [105]:
display(df[6])

,TimeStamp,mains_1,mains_2,kitchen_outlets_3,washer_dryer_4,stove_5,electronics_6,bathroom_gfi_7,refrigerator_8,dishwaser_9,...,air_conditioning_16,air_conditioning_17,Year,Month,Day,Hour,Minute,Seconds,Holiday,Peak Hour
0,2011-05-21 19:39:51,0.0,255.62,5.0,2.0,3.0,0.0,3.0,1.0,0.0,...,0.0,0.0,2011.0,5.0,21.0,19.0,39.0,51.0,1.0,0.0
1,2011-05-21 19:39:57,0.0,257.48,5.0,3.0,2.0,0.0,3.0,0.0,0.0,...,0.0,0.0,2011.0,5.0,21.0,19.0,39.0,57.0,1.0,0.0
2,2011-05-21 19:40:00,0.0,255.07,5.0,3.0,2.0,0.0,4.0,0.0,0.0,...,0.0,0.0,2011.0,5.0,21.0,19.0,40.0,0.0,1.0,0.0
3,2011-05-21 19:40:04,0.0,255.78,5.0,2.0,3.0,0.0,3.0,0.0,0.0,...,0.0,0.0,2011.0,5.0,21.0,19.0,40.0,4.0,1.0,0.0
4,2011-05-21 19:40:07,0.0,256.38,5.0,3.0,2.0,0.0,3.0,0.0,0.0,...,0.0,0.0,2011.0,5.0,21.0,19.0,40.0,7.0,1.0,0.0
5,2011-05-21 19:40:13,0.0,262.64,5.0,3.0,3.0,0.0,3.0,0.0,0.0,...,0.0,0.0,2011.0,5.0,21.0,19.0,40.0,13.0,1.0,0.0
6,2011-05-21 19:40:20,0.0,255.98,5.0,3.0,2.0,0.0,4.0,0.0,0.0,...,0.0,0.0,2011.0,5.0,21.0,19.0,40.0,20.0,1.0,0.0
7,2011-05-21 19:40:27,0.0,255.83,5.0,3.0,2.0,0.0,3.0,0.0,0.0,...,0.0,0.0,2011.0,5.0,21.0,19.0,40.0,27.0,1.0,0.0
8,2011-05-21 19:40:33,0.0,254.32,5.0,2.0,3.0,0.0,3.0,0.0,0.0,...,0.0,0.0,2011.0,5.0,21.0,19.0,40.0,33.0,1.0,0.0
9,2011-05-21 19:40:42,0.0,255.10,5.0,3.0,2.0,0.0,3.0,0.0,0.0,...,0.0,0.0,2011.0,5.0,21.0,19.0,40.0,42.0,1.0,0.0
